In [1]:
# Commented out IPython magic to ensure Python compatibility.
%%capture
import numpy as np
import re
import pandas as pd
!pip install nltk
!pip install summa
!pip install wordninja
from summa import keywords
import re
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')
import wordninja as ninja
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize,sent_tokenize
!pip install sentence_transformers
from sentence_transformers import SentenceTransformer

def clean(query):
  query = re.sub("[^a-zA-Z :\.]", "",query)                         # remove all symbols except .
  
  new_word = []
  for word in nltk.word_tokenize(query):
    if word.lower().strip() != "cohesity": 
      new_word.append(word.strip().lower())
  
  if len(new_word) > 2:
    return ' '.join(new_word)
  return False


def ninja_technique(query):
  return " ".join(ninja.split(query))


wordnet_lemmatizer = WordNetLemmatizer()
def lemmatize(query):
    new_word = []
    for word in query.split():
        new_word.append(wordnet_lemmatizer.lemmatize(word, pos="v").strip())

    return ' '.join(new_word)
 

ps = PorterStemmer()
def stem_(query):
    new_word = []
    for word in query.split():
        new_word.append(ps.stem(word).strip())

    return ' '.join(new_word)


In [3]:
"""Total Case Created     No clicks case creation"""
no_clicks = pd.read_excel("Cohesity Dataset to be worked upon.xlsx",sheet_name="Case Created", usecols=["Activity Detail"])
print(len(no_clicks))
no_clicks.drop_duplicates(inplace=True)
print(len(no_clicks))


3717
3717


In [4]:
case_ids = []
for row in no_clicks["Activity Detail"].to_list():
  id = row[-8:]
  if id.isdigit():
    case_ids.append(row[-8:])
  else:
    case_ids.append("-")

no_clicks["CASE_ID"] = case_ids

# lower + remove word length <=2 
no_clicks["Activity Detail"] = no_clicks["Activity Detail"].apply(clean)   
no_clicks = no_clicks[no_clicks["Activity Detail"] != False]
print(len(no_clicks))

# split joined words 
no_clicks["clean"] = no_clicks["Activity Detail"].apply(ninja_technique)
print(len(no_clicks))

3437
3437


In [5]:
# sample = no_clicks.sample(1000,random_state=694)
sample = no_clicks

sample


,Activity Detail,CASE_ID,clean
0,remove sql server from backup,00848226,remove sql server from backup
1,no errors sql database protection group not ge...,00848166,no errors sql database protection group not ge...
2,sensor reading critical,00848124,sensor reading critical
3,helios scheduled reports not being received co...,00848122,helios scheduled reports not being received co...
4,ce cecece diskmissing slerts codes ce ce,00848113,ce cece ce disk missing s le rts codes ce ce
...,...,...,...
3712,no error epic backup are taking longer time to...,00794944,no error epic backup are taking longer time to...
3713,unable to server register agent version mismatch,00794942,unable to server register agent version mismatch
3714,about the amount of data transferred written d...,00794618,about the amount of data transferred written d...
3715,ce ce vmoptimizedntfsvolumeindexingskipped,00794631,ce ce vm optimized ntfs volume indexing skipped


In [6]:

model = SentenceTransformer('all-mpnet-base-v2')
# model = SentenceTransformer('bert-base-nli-mean-tokens')
encoding_ = model.encode(sample['clean'].to_list())
data_encoding = np.array(encoding_)
np.save('encoding.npy', data_encoding)
embedding = np.load('encoding.npy')

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/363 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [8]:
# Commented out IPython magic to ensure Python compatibility.
%%capture
!pip install nmslib
import nmslib

class NMSLIBIndex():
  def __init__(self, vectors, labels):
    self.dimention = vectors.shape[0]
    self.vectors = vectors.astype('float32')
    self.labels = labels
  
  def build(self):
    self.index = nmslib.init(method='hnsw', space='cosinesimil')
    self.index.addDataPointBatch(self.vectors)
    self.index.createIndex({'post': 2})
  
  # find 50% nearest neigbours (half to the dataset)
  def query(self, vector, k):
    indices = self.index.knnQuery(vector, k=k)
    return [self.labels[i] for i in indices[0]], [round(i,2) for i in indices[1]]

vector_list = []
for emb in embedding:
  vector_list.append(emb)

vect = np.array(vector_list)

new_df = pd.DataFrame({"vector" : vector_list, "title" : sample["clean"], "CASE_ID":sample["CASE_ID"]})
new_df = new_df.reset_index()
new_df.drop(columns=["index"],inplace=True)
new_df

new_df[new_df["title"] == "ce lond bk ch frequent process restarts"]["CASE_ID"].values[0]

nm_index = NMSLIBIndex(vect, new_df["title"])
nm_index.build()

K = embedding.shape[0]//2

pd_dict = {
    "main_title" : [],
    "similar_titles" : [],
    "distance" : [],
    "group" : [],
    "case" : []
}
group = 0

for vector,label in zip(new_df['vector'],new_df['title']) :
  queries_nm, distances_nm = nm_index.query(vector,K) 
  # small_list = []                                        # put all same queries in this list then we take common word as group  
  for q,d in zip(queries_nm, distances_nm):
    # if label != q:
    if d < 0.2:
      pd_dict["main_title"].append(label)
      pd_dict["similar_titles"].append(q)
      pd_dict["distance"].append(d)
      pd_dict["group"].append(group)
      case = new_df[new_df["title"] == q]["CASE_ID"].values[0]
      pd_dict["case"].append(case)

    # small_list.append(q)
  # find common words in this group of same queries  
  print(group,"--------")
  # group = "_".join(set(" ".join(keywords_list).split()))
  group +=1

frame = pd.DataFrame(pd_dict)
frame.drop_duplicates(subset="similar_titles",inplace=True)
frame.to_excel("(k)Total_cases_cohesity_with_caseID(more_similar).xlsx")
frame.head(50)

In [10]:
frame.head(50)

,main_title,similar_titles,distance,group,case
0,remove sql server from backup,remove sql server from backup,0.00,0,00848226
1,no errors sql database protection group not ge...,no errors sql database protection group not ge...,0.00,1,00848166
2,sensor reading critical,sensor reading critical,0.00,2,00848124
3,helios scheduled reports not being received co...,helios scheduled reports not being received co...,0.00,3,00848122
4,ce cece ce disk missing s le rts codes ce ce,ce cece ce disk missing s le rts codes ce ce,0.00,4,00848113
5,free up disk space,free up disk space,0.00,5,00848092
6,f and f we are seeing alerts on one of our nod...,f and f we are seeing alerts on one of our nod...,0.00,6,00848088
7,restoring oracle archive logs using oracle ada...,restoring oracle archive logs using oracle ada...,0.00,7,00848079
8,remove co he s it y rich filed nodes,remove co he s it y rich filed nodes,0.00,8,00848058
9,iam cloud formation question,iam cloud formation question,0.00,9,00848044


In [9]:
# doc2vec

frame.groupby(by=["group"],as_index=False).count().sort_values(by="distance",ascending=False)

frame["similar_titles"].nunique()

len(frame[frame["group"] == 12])

len(frame)

(len(frame[frame["group"] == 12])/len(frame)) * 100

"""

top group (12) has 93 similar queries inside, which is 1.92% of total 4840 rows

"""

for g in set(frame["group"].to_list()):
  queries_in_group = len(frame[frame["group"] == g])
  percentage = (queries_in_group/len(frame)) * 100
  if percentage > 0.75:
    total = len(frame)
    print(f"Group - {g}, has {queries_in_group} queries in it, which is {round(percentage,2)}% of the total {total} queries.")

